In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [3]:
data_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [4]:
img = np.reshape(data_df.iloc[0][1:].values,(28,28))

In [5]:
data_df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
class MnistDataset(Dataset):
    def __init__(self,data_df):
        super().__init__()
        self.data_df = data_df
    def __len__(self):
        return self.data_df.shape[0]
    def __getitem__(self,idx):
        img = np.reshape(self.data_df.iloc[idx][1:].values,(1,28,28)).astype("float32")
        img /= 255.
        target = self.data_df.iloc[idx][0]
        return img,target

In [7]:
from sklearn.model_selection import train_test_split


In [8]:
train_df,test_df = train_test_split(data_df,test_size = 0.2,random_state = 666)

In [9]:
dataset_train = MnistDataset(train_df)
dataloader_train = DataLoader(dataset_train,batch_size=16,shuffle=True)

dataset_test = MnistDataset(test_df)
dataloader_test = DataLoader(dataset_test,batch_size=16,shuffle=False)

In [10]:
for x,y in dataloader_train:
    break

In [11]:
x.shape

torch.Size([16, 1, 28, 28])

In [12]:
import torch.nn.functional as F

In [13]:
class GovnoModel(torch.nn.Module):
    def __init__(self):
        super(GovnoModel,self).__init__()
        self.conv1 = torch.nn.Conv2d(1,8,(3,3))
        self.conv2 = torch.nn.Conv2d(8,16,(3,3))
        self.conv3 = torch.nn.Conv2d(16,32,(3,3))
        self.gap = torch.nn.AdaptiveAvgPool2d(1)
        self.output = torch.nn.Linear(32,10)
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.gap(x)
        x = x.flatten(1)
        x = self.output(x)
        return x

In [14]:
print(torch.cuda.is_available())

True


In [15]:
model = GovnoModel().cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

In [16]:
from tqdm.notebook import tqdm

In [17]:
for e in range(15):
    train_losses = []
    test_losses = []
    for x,y in tqdm(dataloader_train):
        optimizer.zero_grad()
        out = model(torch.tensor(x).float().cuda())
        loss = loss_fn(out,torch.tensor(y).cuda())
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    for x,y in tqdm(dataloader_test):
        out = model(torch.tensor(x).float().cuda())
        loss = loss_fn(out,torch.tensor(y).cuda())
        test_losses.append(loss.item())
    print("train_losses: ",np.mean(train_losses))
    print("test_losses: ",np.mean(test_losses))

  0%|          | 0/2100 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


  0%|          | 0/525 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == "":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


train_losses:  2.191745426881881
test_losses:  2.0144000405357


...

  0%|          | 0/525 [00:00<?, ?it/s]

train_losses:  1.2601697960921696
test_losses:  1.247521759668986


  0%|          | 0/2100 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

train_losses:  1.212238544992038
test_losses:  1.2058903644198462


In [18]:
all_predictions = []
all_targets = []
for x,y in tqdm(dataloader_test):
    out = model(torch.tensor(x).float().cuda())
    all_predictions.append(out.detach().cpu().numpy())
    all_targets.append(y)

  0%|          | 0/525 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [19]:
all_predictions = np.concatenate(all_predictions)
all_targets = np.concatenate(all_targets)

In [20]:
from sklearn.metrics import classification_report, confusion_matrix

In [21]:
print(classification_report(all_targets,np.argmax(all_predictions,axis=1)))

              precision    recall  f1-score   support

           0       0.56      0.62      0.59       824
           1       0.83      0.94      0.88       922
           2       0.40      0.54      0.46       815
           3       0.63      0.33      0.43       873
           4       0.59      0.53      0.56       806
           5       0.51      0.56      0.53       777
           6       0.65      0.74      0.69       824
           7       0.79      0.86      0.82       882
           8       0.64      0.51      0.57       828
           9       0.58      0.51      0.54       849

    accuracy                           0.62      8400
   macro avg       0.62      0.61      0.61      8400
weighted avg       0.62      0.62      0.61      8400



In [22]:
plt.figure(figsize=(15,15))
plt.matshow(confusion_matrix(all_targets,np.argmax(all_predictions,axis=1)),cmap='hot')

NameError: name 'plt' is not defined

In [ ]:
import matplotlib.pyplot as plt